In [54]:
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials
import requests
import json
from keplergl import KeplerGl

In [37]:
def read_google_spreadsheet_to_dataframe(cred_json_path, spreadsheet_id, sheet_name):
    try:
        # Cargamos las credenciales desde el JSON y establecemos el alcance
        scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
        credentials = ServiceAccountCredentials.from_json_keyfile_name(cred_json_path, scope)
        
        
        # Autenticación y acceso a la hoja de cálculo
        client = gspread.authorize(credentials)
        spreadsheet = client.open_by_key(spreadsheet_id)
        sheet = spreadsheet.worksheet(sheet_name)
        
        # Obtenemos los datos como una lista de diccionarios
        data = sheet.get_all_records()
        
        # Creamos un DataFrame de Pandas a partir de los datos
        df = pd.DataFrame(data)
        
        return df
        
    except Exception as e:
        print("Error:", e)
        return None

In [38]:
datos = read_google_spreadsheet_to_dataframe('../../creds/tsc20241-436644aacdef.json', 
'19K2iFwzp2ncUikV0TqrSCQmfDOCCQhqcmQe8q82BdWc', 'Respuestas de formulario 1')

In [39]:
datos.head()

,Marca temporal,Pon la dirección completa de tu lugar favorito
0,29/8/2023 18:51:16,"Carr. Picacho-Ajusco Km 1.5, Jardines del Ajus..."
1,29/8/2023 18:51:35,"Av. Río Churubusco 79, Country Club Churubusco..."
2,29/8/2023 18:51:40,"Av. Ejército Nacional Mexicano 843-B, Granada,..."
3,29/8/2023 18:51:51,"Plan de San Luis 50, La Escalera, Gustavo A. M..."
4,29/8/2023 18:52:06,"Av Miguel Ángel de Quevedo 227, Oxtopulco, Co..."


In [41]:
def get_lat_lng_from_address(address):
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    
    params = {
        "address": address,
        "key": json.load(open("../../creds/maps.json",'r'))['key']
    }
    
    response = requests.get(base_url, params=params)
    data = response.json()
  
    if data["status"] == "OK":
        location = data["results"][0]["geometry"]["location"]
        latitude = location["lat"]
        longitude = location["lng"]
        return latitude, longitude
    else:
        return None



In [44]:
type(datos[''])

pandas.core.series.Series

In [45]:
datos['coord'] = datos['Pon la dirección completa de tu lugar favorito'].map(get_lat_lng_from_address)

In [46]:
datos.head()

,Marca temporal,Pon la dirección completa de tu lugar favorito,coord
0,29/8/2023 18:51:16,"Carr. Picacho-Ajusco Km 1.5, Jardines del Ajus...","(19.2895508, -99.2175273)"
1,29/8/2023 18:51:35,"Av. Río Churubusco 79, Country Club Churubusco...",None
2,29/8/2023 18:51:40,"Av. Ejército Nacional Mexicano 843-B, Granada,...","(19.4390565, -99.2019335)"
3,29/8/2023 18:51:51,"Plan de San Luis 50, La Escalera, Gustavo A. M...","(19.5123919, -99.1394754)"
4,29/8/2023 18:52:06,"Av Miguel Ángel de Quevedo 227, Oxtopulco, Co...",None


In [48]:
limpio = datos.dropna().reset_index(drop=True)

In [50]:
limpio[['lat','lon']] = limpio['coord'].apply(pd.Series)

In [51]:
limpio

,Marca temporal,Pon la dirección completa de tu lugar favorito,coord,lat,lon
0,29/8/2023 18:51:16,"Carr. Picacho-Ajusco Km 1.5, Jardines del Ajus...","(19.2895508, -99.2175273)",19.289551,-99.217527
1,29/8/2023 18:51:40,"Av. Ejército Nacional Mexicano 843-B, Granada,...","(19.4390565, -99.2019335)",19.439056,-99.201933
2,29/8/2023 18:51:51,"Plan de San Luis 50, La Escalera, Gustavo A. M...","(19.5123919, -99.1394754)",19.512392,-99.139475
3,29/8/2023 18:52:12,"Av. de los Insurgentes Sur S/N, C.U., Coyoacán...","(19.3222546, -99.1884774)",19.322255,-99.188477
4,29/8/2023 18:52:20,"Blvd. Miguel de Cervantes Saavedra 386, Grana...","(19.4399546, -99.20509229999999)",19.439955,-99.205092
5,29/8/2023 18:52:22,"Carretera Picacho-Ajusco Km 1.5, Jardines del ...","(19.2895508, -99.2175273)",19.289551,-99.217527
6,29/8/2023 18:52:26,"Restaurante La Oveja Negra, C. Sabino 213, Sta...","(19.4516422, -99.1593863)",19.451642,-99.159386
7,29/8/2023 18:52:37,"Shine Bar Circuito, Avenida Rio Consulado, Tla...","(19.4590909, -99.16332919999999)",19.459091,-99.163329
8,29/8/2023 18:52:40,"Vasco de Quiroga 3880-Nivel 3, Lomas de Santa ...","(19.3613785, -99.2775831)",19.361379,-99.277583
9,29/8/2023 18:52:53,"Cto Centro Comercial 2251, Cd. Satélite, 53100...","(19.5103999, -99.2338814)",19.510400,-99.233881


In [55]:
mapa = KeplerGl(height=500,widht=700)
mapa.add_data(data=limpio[['lat','lon']], name="lugares")

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [58]:
mapa.save_to_html(file_name="mapa.html")

Map saved to mapa.html!
